In [1]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.append('/Users/yuxiangyu/Documents/毕业论文-20171126/毕业论文code/')
from DataAnalysis import DataAnalysis

In [2]:
stat = DataAnalysis()
now_time = stat.get_time()

In [3]:
# 当前时间
now_time

'20180205135430'

## 数据路径

In [4]:
# 获取数据集路径
train_data_path_list = stat.datasets

In [6]:
for ix, tmp_data_path in enumerate(train_data_path_list):
    # 获取数据集名称
    file_name = tmp_data_path.split('/')[-1].split('.')[0]
    print(ix, file_name)

0 iris
1 new-winequality-white
2 new-winequality-red
3 new1-new-glass
4 seeds_dataset
5 new-breast-cancer-wisconsin
6 new-win
7 new-balance-scale
8 new-segmentation


### 读取数据

In [168]:
data_df = stat.open_csv(tmp_data_path)

In [169]:
# 相关性
data_df.corr()

,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,label
col_1,1.000000,-0.273732,0.156794,-0.069809,-0.266087,-0.275442,0.326603,-0.241346,0.502898
col_2,-0.273732,1.000000,-0.481799,-0.165927,0.005396,-0.443750,-0.492262,0.083060,-0.744993
col_3,0.156794,-0.481799,1.000000,-0.005524,0.325958,-0.259592,0.479404,-0.074402,0.598829
col_4,-0.069809,-0.165927,-0.005524,1.000000,-0.193331,-0.208732,-0.102151,-0.094201,0.151565
col_5,-0.266087,0.005396,0.325958,-0.193331,1.000000,-0.317836,-0.042618,-0.007719,-0.010054
col_6,-0.275442,-0.443750,-0.259592,-0.208732,-0.317836,1.000000,-0.112841,0.124968,0.000952
col_7,0.326603,-0.492262,0.479404,-0.102151,-0.042618,-0.112841,1.000000,-0.058692,0.575161
col_8,-0.241346,0.083060,-0.074402,-0.094201,-0.007719,0.124968,-0.058692,1.000000,-0.188278
label,0.502898,-0.744993,0.598829,0.151565,-0.010054,0.000952,0.575161,-0.188278,1.000000


In [171]:
# 行列数量
row_len, col_len = data_df.shape

### 训练集和测试集

In [172]:
train_data, test_data = stat.get_train_test(data_df)

### 中心权重 迭代权重求解

In [175]:
orginal_weight_array = np.array(list([1]* (col_len - 1)))

In [176]:
orginal_weight_array

array([1, 1, 1, 1, 1, 1, 1, 1])

In [244]:
# orginal_weight_array = np.random.random(col_len - 1)
orginal_weight_array = orginal_weight_array/sum(orginal_weight_array) # 归一化
center_result_df = pd.DataFrame(columns=train_data.columns)
iter_center_result_df = pd.DataFrame(columns=train_data.columns)
for class_name, values in train_data.groupby('label'):
    tmp_class_name_list = values.columns
    columns_name_list = [i for i in tmp_class_name_list if 'label' not in i]
    tmp_center_mean_list = list(values[columns_name_list].mean())
    tmp_center_mean_list.append(class_name)
    center_result_df.loc[class_name, :] = tmp_center_mean_list
    tmp_result,iter_n = stat.get_weights_iteration(values[columns_name_list], orginal_weight_array, error_param = 1e-4, max_iter= 1000, alpha=0.5)
    tmp_result.append(class_name)
    iter_center_result_df.loc['{name}:{iter}'.format(name = class_name, iter = iter_n), :] = tmp_result
    

In [245]:
# 中心点
center_result_df

,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,label
1,13.1312,3.56442,1.18047,72.6579,0.471395,8.82605,0.00465116,0.0493023,1
2,13.0849,3.014,1.40109,72.6391,0.524364,9.04491,0.0645455,0.0807273,2
3,13.591,3.538,1.233,72.212,0.428,8.814,0,0.063,3
5,13.21,0.647143,2.11,72.0757,1.42714,10.0686,0.314286,0,5
6,14.266,2.002,1.554,72.536,0,9.574,0,0,6
7,14.3368,0.377895,2.16632,73.0884,0.366842,8.54684,1.04368,0.0157895,7


In [246]:
# 迭代后权重
iter_center_result_df

,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,label
1:1001,0.0294118,0.0588235,0.0588235,0.0294118,0.0588235,0.0294118,0.588235,0.147059,1
2:1001,0.08,0.04,0.16,0.04,0.16,0.02,0.1,0.4,2
3:67,0.000194951,0.000389902,0.000194951,9.74754e-05,0.000389902,9.74754e-05,0.998148,0.000487377,3
5:81,9.76169e-05,4.88085e-05,9.76169e-05,4.88085e-05,2.44042e-05,2.44042e-05,6.10106e-05,0.999597,5
6:43,0.000121951,0.000121951,0.000243902,0.000243902,0.062439,0.000243902,0.312195,0.62439,6
7:1001,0.036036,0.036036,0.0720721,0.036036,0.036036,0.018018,0.045045,0.720721,7


In [247]:
iter_without_label = iter_center_result_df[list(iter_center_result_df.columns)[:-1]]

In [248]:
iter_without_label.std()

col_1    0.031679
col_2    0.025700
col_3    0.063402
col_4    0.019477
col_5    0.059049
col_6    0.012835
col_7    0.388769
col_8    0.373077
dtype: float64

In [249]:
iter_without_label.mean()

col_1    0.024310
col_2    0.022570
col_3    0.048572
col_4    0.017640
col_5    0.052952
col_6    0.011299
col_7    0.340614
col_8    0.482042
dtype: float64

## 传统 KNN 算法

In [250]:
from sklearn import neighbors as knn

In [251]:
train_array = train_data.drop('label', axis=1)
test_array = test_data.drop('label', axis=1)

In [252]:
for k_value in range(1, 21):
    clf = knn.KNeighborsClassifier(n_neighbors = k_value, weights = 'uniform')
    clf.fit(train_data.drop('label', axis = 1), train_data['label'])
    # 预测结果
    predict_label_array = clf.predict(test_array)
    classic_knn_result = stat.get_accuracy_score(predict_label_array, test_data['label'])
    print('k值:{kvalue}, 分类准确率:{re:.2f} %'.format(kvalue = k_value,re = classic_knn_result * 100)) 
    

k值:1, 分类准确率:66.67 %
k值:2, 分类准确率:58.67 %
k值:3, 分类准确率:56.00 %
k值:4, 分类准确率:60.00 %
k值:5, 分类准确率:56.00 %
k值:6, 分类准确率:50.67 %
k值:7, 分类准确率:52.00 %
k值:8, 分类准确率:50.67 %
k值:9, 分类准确率:50.67 %
k值:10, 分类准确率:49.33 %
k值:11, 分类准确率:52.00 %
k值:12, 分类准确率:52.00 %
k值:13, 分类准确率:52.00 %
k值:14, 分类准确率:53.33 %
k值:15, 分类准确率:52.00 %
k值:16, 分类准确率:53.33 %
k值:17, 分类准确率:52.00 %
k值:18, 分类准确率:50.67 %
k值:19, 分类准确率:50.67 %
k值:20, 分类准确率:53.33 %


## 剔除列计算

In [253]:
tmp_iter_std = iter_without_label.std()

In [254]:
tmp_iter_std.sort_values(inplace= True)

In [255]:
tmp_iter_std

col_6    0.012835
col_4    0.019477
col_2    0.025700
col_1    0.031679
col_5    0.059049
col_3    0.063402
col_8    0.373077
col_7    0.388769
dtype: float64

In [256]:
center_result_df = center_result_df[list(center_result_df.columns)[:-1]]

In [257]:
center_result_df_std = center_result_df.std()

In [258]:
center_result_df_std.sort_values(inplace= True)

In [259]:
center_result_df_std

col_8    0.034346
col_4    0.359858
col_7    0.412982
col_3    0.431939
col_5    0.474556
col_6    0.568529
col_1    0.569866
col_2    1.420182
dtype: float64

In [260]:
# del_col = ['col_4','col_6', 'col_7'] # wine
del_col = list(tmp_iter_std.iloc[:2].index)

In [261]:
del_col

['col_6', 'col_4']

In [262]:
# 基于中心结果删除
# del_col = list(center_result_df_std.iloc[:3].index)

In [263]:
del_train_data = train_data.drop(del_col, axis=1)
del_test_data = test_data.drop(del_col, axis=1)

In [264]:
del_train_data.head(3)

,col_1,col_2,col_3,col_5,col_7,col_8,label
136,13.00,3.80,1.08,0.56,0.00,0.12,2
206,14.94,0.00,1.87,0.00,1.38,0.00,7
75,13.02,3.58,1.51,0.69,0.00,0.00,2


In [265]:
(iter_center_result_df.T).loc[del_col].sum()

1:1001    0.058824
2:1001    0.060000
3:67      0.000195
5:81      0.000073
6:43      0.000488
7:1001    0.054054
dtype: float64

In [266]:
del_test_data.head(3)

,col_1,col_2,col_3,col_5,col_7,col_8,label
161,13.64,3.54,0.75,0.16,0.15,0.24,3
120,13.25,3.76,1.32,0.58,0.00,0.00,2
105,11.45,0.00,1.88,0.81,0.00,0.34,2


In [267]:
for del_k_value in range(1, 21):
    del_clf = knn.KNeighborsClassifier(n_neighbors = del_k_value, weights = 'uniform')
    del_clf.fit(del_train_data.drop('label', axis = 1), del_train_data['label'])
    # 预测结果
    del_predict_label_array = del_clf.predict(del_test_data.drop('label', axis=1))
    del_classic_knn_result = stat.get_accuracy_score(del_predict_label_array, del_test_data['label'])
    print('del_k 值:{kvalue}, 分类准确率:{re:.2f} %'.format(kvalue = del_k_value,re = del_classic_knn_result * 100)) 
    

del_k 值:1, 分类准确率:53.33 %
del_k 值:2, 分类准确率:54.67 %
del_k 值:3, 分类准确率:48.00 %
del_k 值:4, 分类准确率:52.00 %
del_k 值:5, 分类准确率:52.00 %
del_k 值:6, 分类准确率:56.00 %
del_k 值:7, 分类准确率:54.67 %
del_k 值:8, 分类准确率:54.67 %
del_k 值:9, 分类准确率:52.00 %
del_k 值:10, 分类准确率:56.00 %
del_k 值:11, 分类准确率:58.67 %
del_k 值:12, 分类准确率:60.00 %
del_k 值:13, 分类准确率:58.67 %
del_k 值:14, 分类准确率:60.00 %
del_k 值:15, 分类准确率:60.00 %
del_k 值:16, 分类准确率:57.33 %
del_k 值:17, 分类准确率:56.00 %
del_k 值:18, 分类准确率:58.67 %
del_k 值:19, 分类准确率:53.33 %
del_k 值:20, 分类准确率:50.67 %
